In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('insurance_pre.csv')

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset['charges']

In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 15, 20, 30, 50],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 4, 10, 20],
    'max_features': [None, 'auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 50, 100],
    'min_impurity_decrease': [0.0, 0.01, 0.1],
    'ccp_alpha': [0.0, 0.01, 0.1]
}
grid = GridSearchCV(DecisionTreeRegressor(), param_grid, refit = True, verbose = 3 ,n_jobs=-1)
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 252000 candidates, totalling 1260000 fits


C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
315000 fits failed out of a total of 1260000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
197514 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constra

GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 0.01, 0.1],
                         'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': [None, 5, 10, 15, 20, 30, 50],
                         'max_features': [None, 'auto', 'sqrt', 'log2'],
                         'max_leaf_nodes': [None, 10, 20, 50, 100],
                         'min_impurity_decrease': [0.0, 0.01, 0.1],
                         'min_samples_leaf': [1, 2, 4, 10, 20],
                         'min_samples_split': [2, 5, 10, 20, 50],
                         'splitter': ['best', 'random']},
             verbose=3)

In [10]:
from sklearn.model_selection import GridSearchCV

# Assuming 'grid' is your GridSearchCV object and has been fitted to the data
print("Best parameters:", grid.best_params_)

# Access the mean test score for the best parameters
best_index = grid.best_index_
best_r2_score = grid.cv_results_['mean_test_score'][best_index]
# Printing the best R² score
print("r2:",best_r2_score)

Best parameters: {'ccp_alpha': 0.0, 'criterion': 'absolute_error', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 4, 'min_samples_split': 10, 'splitter': 'best'}
r2: 0.830056098330347


In [11]:
table=pd.DataFrame.from_dict(grid.cv_results_)

In [12]:
table.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ccp_alpha,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_impurity_decrease,...,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
129474,0.021853,0.007664,0.000000,0.000000,0.01,absolute_error,5,None,50,0.0,...,best,"{'ccp_alpha': 0.01, 'criterion': 'absolute_err...",0.821535,0.811995,0.875253,0.845989,0.795508,0.830056,0.027889,1
213474,0.020135,0.008502,0.001300,0.002599,0.1,absolute_error,5,None,50,0.0,...,best,"{'ccp_alpha': 0.1, 'criterion': 'absolute_erro...",0.821535,0.811995,0.875253,0.845989,0.795508,0.830056,0.027889,1
45124,0.014653,0.000969,0.001654,0.000617,0.0,absolute_error,5,None,None,0.1,...,best,"{'ccp_alpha': 0.0, 'criterion': 'absolute_erro...",0.821535,0.811995,0.875253,0.845989,0.795508,0.830056,0.027889,1
129674,0.018748,0.006289,0.000000,0.000000,0.01,absolute_error,5,None,100,0.01,...,best,"{'ccp_alpha': 0.01, 'criterion': 'absolute_err...",0.821535,0.811995,0.875253,0.845989,0.795508,0.830056,0.027889,1
213674,0.015625,0.000008,0.003123,0.006246,0.1,absolute_error,5,None,100,0.01,...,best,"{'ccp_alpha': 0.1, 'criterion': 'absolute_erro...",0.821535,0.811995,0.875253,0.845989,0.795508,0.830056,0.027889,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205231,0.000000,0.000000,0.000000,0.000000,0.1,friedman_mse,30,auto,50,0.0,...,random,"{'ccp_alpha': 0.1, 'criterion': 'friedman_mse'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,189001
205232,0.000000,0.000000,0.000000,0.000000,0.1,friedman_mse,30,auto,50,0.0,...,best,"{'ccp_alpha': 0.1, 'criterion': 'friedman_mse'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,189001
205233,0.003101,0.006202,0.000000,0.000000,0.1,friedman_mse,30,auto,50,0.0,...,random,"{'ccp_alpha': 0.1, 'criterion': 'friedman_mse'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,189001
205235,0.000000,0.000000,0.000000,0.000000,0.1,friedman_mse,30,auto,50,0.0,...,random,"{'ccp_alpha': 0.1, 'criterion': 'friedman_mse'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,189001


In [ ]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

In [ ]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))